# Eat Safe, Love

## Notebook Set Up

In [37]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [38]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [39]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [40]:
# review the collections in our database
collections = db.list_collection_names()

print("Collections in the 'uk_food' database:")
for collection in collections:
    print(collection)

Collections in the 'uk_food' database:
establishments


In [41]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
#query =
# Use count_documents to display the number of documents in the result
# Display the first document in the results using pprint

# Define the query to find establishments with a hygiene score of 20
query = {
    "scores.Hygiene": 20  # Assuming the field name is "Hygiene" for the hygiene score
}

# Use the count_documents method to count the number of matching documents
count = establishments.count_documents(query)

# Print the number of documents that match the query
print(f"Number of establishments with a hygiene score of 20: {count}")

# Use the find method to retrieve the first document in the results
result = establishments.find_one(query)

# Display the first document in the results using pprint
if result:
    print("First establishment with a hygiene score of 20:")
    pprint(result)
else:
    print("No establishments found with a hygiene score of 20.")


    


Number of establishments with a hygiene score of 20: 41
First establishment with a hygiene score of 20:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6510d0cd2a220487fa93d8f4'),
 'geocode': {'latitude': {'$convert': {'input': '$geocode.latitude',
                                       'to': 'decimal'}},
             '

In [7]:
# Convert the result to a Pandas DataFrame
result_list = list(establishments.find(query))

df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
row_count = df.shape[0]
print("Number of rows in the DataFrame:", row_count)

# Display the first 10 rows of the DataFrame
first_10_rows = df.head(10)
print("The first 10 rows of the DataFrame:")
print(first_10_rows)


Number of rows in the DataFrame: 41
The first 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  6510d0cd2a220487fa93d8f4   110681                  0   
1  6510d0cd2a220487fa93dc75   612039                  0   
2  6510d0cd2a220487fa93df7f   730933                  0   
3  6510d0cd2a220487fa93e16d   172735                  0   
4  6510d0cd2a220487fa93e17c   172953                  0   
5  6510d0ce2a220487fa93eb1b   512854                  0   
6  6510d0ce2a220487fa93ed3c  1537089                  0   
7  6510d0ce2a220487fa940267   155648                  0   
8  6510d0cf2a220487fa9406ab  1012883                  0   
9  6510d0cf2a220487fa940ebd   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pi

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
count = db.uk_food.establishments.count_documents(query)

print("Number of establishments in London with RatingValue >= 4:", count)

# Display the first document in the results using pprint
first_document = db.uk_food.establishments.find_one(query)

if first_document:
    print("First establishment in the results:")
    pprint(first_document)
else:
    print("No establishments found that match the criteria.")


Number of establishments in London with RatingValue >= 4: 0
No establishments found that match the criteria.


In [11]:
# Convert the result to a Pandas DataFrame

result = db.uk_food.establishments.find(query)
result_list = list(result)
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame

row_count = df.shape[0]
print("Number of rows in the DataFrame:", row_count)

# Display the first 10 rows of the DataFrame

first_10_rows = df.head(10)
print("The first 10 rows of the DataFrame:")
print(first_10_rows)


Number of rows in the DataFrame: 0
The first 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [36]:
# Define the latitude and longitude for "Penang Flavours" (assuming you have the coordinates)
latitude = 51.490142
longitude = 0.08384
degree_search = 0.01

# Define the query to find establishments with RatingValue equal to 5 and within the specified latitude and longitude range
query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}

# Define the sort criteria to sort by hygiene score in ascending order
sort = [("scores.Hygiene", 1)]

# Limit the query to the top 5 results
results = db.establishments.find(query).sort(sort).limit(5)

# Print the results
for result in results:
    pprint(result)


In [14]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)

# Display the Pandas DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [42]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
# Print the number of documents in the result
# Print the first 10 results

# Define the query to find establishments with a hygiene score of 0
query = {
    "scores.Hygiene": {
        "$eq": 0
    }
}

# Assuming 'db' is the MongoClient instance and 'establishments' is the collection
cursor = db.establishments.find(query)

# Initialize a dictionary to store the counts for each Local Authority
counts = {}

# Iterate through the cursor and count the establishments for each Local Authority
for doc in cursor:
    local_authority = doc["LocalAuthorityName"]
    counts[local_authority] = counts.get(local_authority, 0) + 1

# Convert the dictionary to a list of tuples for sorting
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

# Print the number of documents in the result
print("Number of documents in the result:", len(sorted_counts))

# Print the first 10 results
print("The first 10 results:")
for i, (local_authority, count) in enumerate(sorted_counts[:10]):
    print(f"{i + 1}. Local Authority: {local_authority}, Count: {count}")


Number of documents in the result: 55
The first 10 results:
1. Local Authority: Thanet, Count: 1130
2. Local Authority: Greenwich, Count: 882
3. Local Authority: Maidstone, Count: 713
4. Local Authority: Newham, Count: 711
5. Local Authority: Swale, Count: 686
6. Local Authority: Chelmsford, Count: 680
7. Local Authority: Medway, Count: 672
8. Local Authority: Bexley, Count: 607
9. Local Authority: Southend-On-Sea, Count: 586
10. Local Authority: Tendring, Count: 542


In [44]:
# Convert the cursor to a Pandas DataFrame
df = pd.DataFrame(cursor)

# Display the number of rows in the DataFrame
row_count = len(df)
print("Number of rows in the DataFrame:", row_count)

# Display the first 10 rows of the DataFrame
print("The first 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
The first 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []
